In [1]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.2
    Uninstalling Keras-2.2.2:
      Successfully uninstalled Keras-2.2.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -U tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip show tensorflow

Name: tensorflow
Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\user\anaconda3\lib\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [269]:
import keras
import numpy as np 
import sklearn 
from keras.models import Sequential
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils  # 用來後續將 label 標籤轉為 one-hot-encoding  
from matplotlib import pyplot as plt
import pydotplus
import graphviz
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import seaborn as sns
import pandas as pd
import numpy as np
import csv
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

In [270]:
col_names = ['WhiteKingFile','WhiteKingRank','WhiteRookFile','WhiteRookRank','BlackKingFile','BlackKingRank','Y']
# load datasetSex
data = pd.read_csv("krkopt.csv", header=None, names=col_names)


In [271]:
data.head()

,WhiteKingFile,WhiteKingRank,WhiteRookFile,WhiteRookRank,BlackKingFile,BlackKingRank,Y
0,a,1,b,3,c,2,draw
1,a,1,c,1,c,2,draw
2,a,1,c,1,d,1,draw
3,a,1,c,1,d,2,draw
4,a,1,c,2,c,1,draw


In [272]:
#LabelEncoder更改資料型態
le=LabelEncoder()
for col in data[['WhiteKingFile','WhiteKingRank','WhiteRookFile','WhiteRookRank', 'BlackKingFile','BlackKingRank','Y']]:
    data[col]=le.fit_transform(data[col])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28056 entries, 0 to 28055
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   WhiteKingFile  28056 non-null  int32
 1   WhiteKingRank  28056 non-null  int64
 2   WhiteRookFile  28056 non-null  int32
 3   WhiteRookRank  28056 non-null  int64
 4   BlackKingFile  28056 non-null  int32
 5   BlackKingRank  28056 non-null  int64
 6   Y              28056 non-null  int32
dtypes: int32(4), int64(3)
memory usage: 1.1 MB


In [273]:
#查詢缺失值，並丟棄不需要的屬性欄位
data.replace(" ?", np.nan, inplace= True)
new_data = data.dropna()
print(r'總比數紀錄:{}'.format(data.shape[0]))
print(r'刪除含缺漏紀錄資料後比數:{}'.format(data.dropna().shape[0]))
pd.get_dummies(new_data['WhiteKingFile'])
WhiteKingFile_encoding = pd.get_dummies(new_data['WhiteKingFile'], prefix= 'WhiteKingFile')

pd.get_dummies(new_data['WhiteKingRank'])
WhiteKingRank_encoding = pd.get_dummies(new_data['WhiteKingRank'], prefix= 'WhiteKingRank')
new_data = new_data.drop('WhiteKingRank', 1)

pd.get_dummies(new_data['WhiteRookFile'])
WhiteRookFile_encoding = pd.get_dummies(new_data['WhiteRookFile'], prefix= 'WhiteRookFile')
#new_data = new_data.drop('marital-status', 1)

pd.get_dummies(new_data['WhiteRookRank'])
WhiteRookRank_encoding = pd.get_dummies(new_data['WhiteRookRank'], prefix= 'WhiteRookRank')

pd.get_dummies(new_data['BlackKingFile'])
BlackKingFlie_encoding = pd.get_dummies(new_data['BlackKingFile'], prefix= 'BlackKingFile')
new_data = new_data.drop('BlackKingFile', 1)

pd.get_dummies(new_data['BlackKingRank'])
BlackKingRank_encoding = pd.get_dummies(new_data['BlackKingRank'], prefix= 'BlackKingRank')

pd.get_dummies(new_data['Y'])
Y_encoding = pd.get_dummies(new_data['Y'], prefix= 'Y')


pd.DataFrame(new_data)

總比數紀錄:28056
刪除含缺漏紀錄資料後比數:28056


C:\Users\USER\AppData\Local\Temp\ipykernel_5584\11363774.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  new_data = new_data.drop('WhiteKingRank', 1)
C:\Users\USER\AppData\Local\Temp\ipykernel_5584\11363774.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  new_data = new_data.drop('BlackKingFile', 1)


,WhiteKingFile,WhiteRookFile,WhiteRookRank,BlackKingRank,Y
0,0,1,2,1,0
1,0,2,0,1,0
2,0,2,0,0,0
3,0,2,0,1,0
4,0,2,1,0,0
...,...,...,...,...,...
28051,1,6,6,4,11
28052,1,6,6,5,11
28053,1,6,6,6,11
28054,1,6,6,4,11


In [274]:
#合併 onehot encoding
new_data = pd.concat([new_data['WhiteKingFile'], WhiteKingRank_encoding,WhiteRookFile_encoding,
           WhiteRookRank_encoding,BlackKingFlie_encoding,BlackKingRank_encoding,Y_encoding],axis=1)
new_data

,WhiteKingFile,WhiteKingRank_0,WhiteKingRank_1,WhiteKingRank_2,WhiteKingRank_3,WhiteRookFile_0,WhiteRookFile_1,WhiteRookFile_2,WhiteRookFile_3,WhiteRookFile_4,...,Y_8,Y_9,Y_10,Y_11,Y_12,Y_13,Y_14,Y_15,Y_16,Y_17
0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28051,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
28052,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
28053,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
28054,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [275]:
from tkinter import Y
feature_cols = ['WhiteKingFile','WhiteKingRank','WhiteRookFile','WhiteRookRank','BlackKingFile']
X = data[feature_cols] # Features
y = data.Y # Target variable
y

0         0
1         0
2         0
3         0
4         0
         ..
28051    11
28052    11
28053    11
28054    11
28055    11
Name: Y, Length: 28056, dtype: int32

In [276]:
#正規化
data2 = preprocessing.normalize(new_data, norm= 'l2')
scaler = MinMaxScaler()
data3 = scaler.fit(new_data)
data3 = scaler.transform(new_data)

In [277]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.3,random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5,random_state=42)
x_train.shape

(19639, 4)

In [278]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# 特徵縮放，使用常態化(Normalization)，公式 = (x - min) / (max - min)
# 顏色範圍：0~255，所以，公式簡化為 x / 255
x_train, x_test = x_train / 255.0, x_test / 255.0



In [279]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [280]:
# 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [281]:
# 模型訓練
model.fit(x_train, y_train, epochs=20,batch_size=10)
# 模型評估，打分數
model.evaluate(x_test, y_test)

Epoch 1/20
6000/6000 [==============================] - 8s 1ms/step - loss: 0.2574 - accuracy: 0.9243
Epoch 2/20
6000/6000 [==============================] - 7s 1ms/step - loss: 0.1261 - accuracy: 0.9621
Epoch 3/20
6000/6000 [==============================] - 7s 1ms/step - loss: 0.0986 - accuracy: 0.9690
Epoch 4/20
6000/6000 [==============================] - 7s 1ms/step - loss: 0.0835 - accuracy: 0.9732
Epoch 5/20
6000/6000 [==============================] - 7s 1ms/step - loss: 0.0738 - accuracy: 0.9765
Epoch 6/20
6000/6000 [==============================] - 6s 1ms/step - loss: 0.0659 - accuracy: 0.9789
Epoch 7/20
6000/6000 [==============================] - 6s 1ms/step - loss: 0.0600 - accuracy: 0.9808
Epoch 8/20
6000/6000 [==============================] - 6s 1ms/step - loss: 0.0558 - accuracy: 0.9818
Epoch 9/20
6000/6000 [==============================] - 7s 1ms/step - loss: 0.0522 - accuracy: 0.9833
Epoch 10/20
6000/6000 [==============================] - 7s 1ms/step - loss: 0.049

[0.10109975934028625, 0.9800000190734863]

In [261]:
# 預測測試集資料
y_pred = model.predict(x_test)

313/313 [==============================] - 0s 655us/step


In [262]:
# 將預測結果轉換為類別
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [268]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import random
from sklearn import metrics
# 顯示訓練成果(分數)
scores = model.evaluate(x_test, y_test)
print("MAE:",mean_absolute_error(y_test,y_pred))
print("RMSE : %.4g" % mean_squared_error(y_test, y_pred,squared=False))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred))

true = [random.randint(1, 2) for _ in range(10)]
prediction = [1 for _ in range(9)]
prediction.append(2)
print('True:', true)
print('Pred:', prediction)
print('Precision:', metrics.precision_score(true, prediction))
print('Recall:', metrics.recall_score(true, prediction))
print('F1:', metrics.f1_score(true, prediction))


313/313 [==============================] - 0s 911us/step - loss: 0.1174 - accuracy: 0.9803
MAE: 0.076
RMSE : 0.6098
MAPE: 36028797018963.984
True: [2, 1, 2, 2, 1, 2, 1, 2, 1, 2]
Pred: [1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
Precision: 0.4444444444444444
Recall: 1.0
F1: 0.6153846153846153


In [8]:
def FileConvert(s):
    file=['a','b','c','d','e','f','g','h']
    index=0
    for f in range(0,len(file)):
        if s.strip().lower()==file[f].lower():
            index=f+1
    return index

In [9]:
#模型訓練與績效評估
def ModelMain(training,verifying,testing):
    print("Model")
    

In [10]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
dataSetPath=r'dataset\original\krkopt.data'
allData=[]
with open(dataSetPath,encoding="utf-8", newline="") as csvData:
  reader = csv.reader(csvData)
  for r in reader:
    if len(r)>0:
      allData.append([FileConvert(r[0]),int(r[1]),FileConvert(r[2]),int(r[3]),FileConvert(r[4]),int(r[5]),r[6]])
allData=np.array(allData)
allData=pd.DataFrame(allData, columns =['WhiteKingFile','WhiteKingRank','WhiteRookFile','WhiteRookRank','BlackKingFile','BlackKingRank','DepthOfWinForWhite'])
training,testing=train_test_split(allData,test_size=0.15)
trainingT,trainingV=train_test_split(training,test_size=(85*15/100)/100)
trainingData=trainingT
validationData=trainingV
testingData=testing
ModelMain(trainingData,validationData,testingData)


Model
